Code Created by Florent Poux. 
This is part of a joint work with:
* Prof. Sander Oude Elberink, 
* Prof. Mila Koeva, 
* Prof. Ville Lehtola, 
* Prof. Nourian Pirouz, 
* Prof. Paulo Raposo.

Licence MIT, worked done with UTWENTE. Please refer to the article for the full comprehension.

Members of the [ITC UTWENTE](https://www.itc.nl/) or students enrolled in a Program can use the code with privileged IP Rights. 

*   Have fun with this notebook that you can very simply run (ctrl+Enter) !
*   Make sure to modify the paths to your dataset, either locally stored, on the CRIB, or in your Drive.
*   You are free to reuse in your own project, please use the followinf reference if so: XXX

For assistance:  [Linkedin](https://www.linkedin.com/in/florent-poux-point-cloud/), [Medium](https://medium.com/@florentpoux) or [Twitter](https://twitter.com/PouxPointCloud).

Enjoy!

# Step 3: Python Automation

# 3.1. Importing libraries

In [2]:
import sys
import os
#Base libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#3D Libraries
import open3d as o3d
import laspy
print(laspy.__version__)

#GLB Libraries
from pygltflib import GLTF2

#Geospatial libraries
import rasterio
import alphashape as ash
import geopandas as gpd
import shapely as sh

from rasterio.transform import from_origin
from rasterio.enums import Resampling
from rasterio.features import shapes
from shapely.geometry import Polygon

import copy
import time
# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')
import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ

import open3d.core as o3c

if o3d.__DEVICE_API__ == 'cuda':
    import open3d.cuda.pybind.t.pipelines.registration as treg
else:
    import open3d.cpu.pybind.t.pipelines.registration as treg



print(f"Open 3D Version: {o3d.__version__}")

2.5.3
Open 3D Version: 0.18.0


# Helper visualization function

In [3]:
def draw_registration_result(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    #source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

# 3.2. Loading 3D datasets

We define the specific paths to our dataset:

In [251]:
data_folder="../DATA/"
pc_dataset="28GN1_08_sampled.xyz"
mesh_dataset="NL.IMBAG.Pand.0141100000048693.obj"
glb_dataset="model.glb"
result_folder="../DATA/RESULTS/"

We can prepare the point cloud by first creating a Pandas DataFrame object called pcd_df, which will host the point cloud data:

In [ ]:
# pcd_df= pd.read_csv(data_folder+pc_dataset, delimiter=";")
# print(pcd_df.columns)

Numpy to Open3D

In [ ]:
# pcd_o3d=o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.array(pcd_df[['X','Y','Z']])))
# pcd_o3d.colors=o3d.utility.Vector3dVector(np.array(pcd_df[['R','G','B']])/255)
# pcd_o3d

Loading the Mesh dataset (3DBAG)

In [ ]:
mesh=o3d.io.read_triangle_mesh(data_folder+mesh_dataset)
mesh.paint_uniform_color([0.9,0.9,0.9])
mesh.compute_vertex_normals()
print(mesh)
print(mesh.get_center())

In [ ]:
#Convex hull
hull,_=mesh.compute_convex_hull()
hull_ls=o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color([1,0,0])
# o3d.visualization.draw_geometries([mesh,hull_ls], window_name="Convex Hull", width=800, height=600)

Loading a glb model

In [ ]:
glb = o3d.io.read_triangle_mesh(data_folder+glb_dataset)
# Swap y and z coordinates
vertices = np.asarray(glb.vertices)
vertices = vertices[:, [0, 2, 1]]  # Swap y and z coordinates
# Reflect the x-axis to fix the mirroring issue
vertices[:, 0] = -vertices[:, 0]
glb.vertices = o3d.utility.Vector3dVector(vertices)

glb.paint_uniform_color([0.2, 0.2, 0.2])
print(glb)
print("Glb center:")
print(glb.get_center())
glb.compute_vertex_normals()
o3d.visualization.draw_geometries([glb], window_name="GLB Mesh", width=800, height=600)


# 3.3. Python 3D Visualization

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

## Rotation function

In [ ]:
def rotation_matrix(axis, angle, is_degree=True):
    """Generate a rotation matrix given an axis and an angle.
    Args:
        axis (str): 'x', 'y', or 'z' axis to rotate around.
        angle (float): The angle to rotate by.
        is_degree (bool): If True, the angle is in degrees, otherwise radians.

    Returns:
        numpy.ndarray: The rotation matrix.
    """
    if is_degree:
        angle = np.radians(angle)
    c, s = np.cos(angle), np.sin(angle)
    if axis == 'x':
        return np.array([
            [1, 0, 0],
            [0, c, -s],
            [0, s, c]
        ])
    elif axis == 'y':
        return np.array([
            [c, 0, s],
            [0, 1, 0],
            [-s, 0, c]
        ])
    elif axis == 'z':
        return np.array([
            [c, -s, 0],
            [s, c, 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'")

# Aligning GLB model to 3D Bag object

Translation of the 3d object for rough alignment

In [ ]:
# Get the translation matrix
T=np.identity(4)	# Create an identity matrix
T[:3,3]=np.array(mesh.get_center())-np.array(glb.get_center()) # Set the translation values to the difference between the centers of the two point clouds
print(T)
glb_t = copy.deepcopy(glb) # Create a copy of the glb mesh
glb_t.transform(T)  # Apply the translation matrix to the glb mesh

# Draw the registration result
draw_registration_result(mesh,glb_t,np.identity(4))


# Manually rotating the glb to get a rough alignment before icp pipelines

In [ ]:
R =rotation_matrix('z', -16) # Generate a rotation matrix to rotate the glb mesh 170 degrees around the z-axis



# Apply the rotation matrix to the glb mesh
glb_r = copy.deepcopy(glb_t)
glb_r.rotate(R, center=glb_r.get_center())


# Draw the registration result
draw_registration_result(mesh, glb_r, np.eye(4))

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [ ]:
voxel_size = 0.05  # means 5cm for the dataset

# # Convert mesh to point cloud for ICP
mesh_pcd = o3d.geometry.PointCloud()
mesh_pcd.points = mesh.vertices
mesh_pcd.normals = mesh.vertex_normals
mesh_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

source_down, source_fpfh = preprocess_point_cloud(mesh_pcd, voxel_size)

# # Convert GLB mesh to point cloud for ICP
glb_pcd = o3d.geometry.PointCloud()
glb_pcd.points = glb_r.vertices
glb_pcd.normals = glb_r.vertex_normals
glb_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

target_down, target_fpfh = preprocess_point_cloud(glb_pcd, voxel_size)

In [ ]:
draw_registration_result(source_down, target_down, np.identity(4))

In [ ]:
# def execute_global_registration(source_down, target_down, source_fpfh,
#                                 target_fpfh, voxel_size):
#     distance_threshold = voxel_size * 1.5
#     print(":: RANSAC registration on downsampled point clouds.")
#     print("   Since the downsampling voxel size is %.3f," % voxel_size)
#     print("   we use a liberal distance threshold %.3f." % distance_threshold)
#     result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
#         source_down, target_down, source_fpfh, target_fpfh, True,
#         distance_threshold,
#         o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
#         3, [
#             o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
#                 0.9),
#             o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
#                 distance_threshold)
#         ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
#     return result

In [ ]:
# result_ransac = execute_global_registration(source_down, target_down,
#                                             source_fpfh, target_fpfh,
#                                             voxel_size)
# print(result_ransac)
# draw_registration_result(source_down, target_down, result_ransac.transformation)
# print("RANSAC registration result")
# print(result_ransac.transformation)
# draw_registration_result(mesh_pcd, glb_r, result_ransac.transformation)

In [ ]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold,np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

In [ ]:
result_icp = refine_registration(source_down, target_down, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source_down, target_down, result_icp.transformation)
draw_registration_result(mesh, glb_r, result_icp.transformation)

In [ ]:
# Perform ICP fine alignment
threshold = 0.005  # Set a larger threshold if needed
icp_result = o3d.pipelines.registration.registration_icp(
    source_down, target_down, threshold, result_icp.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)
# Transform the GLB model based on ICP result
glb_icp = copy.deepcopy(glb_r)
glb_icp.transform(icp_result.transformation)

# Color the mesh with a beautiful gradient
glb_icp.compute_vertex_normals()
glb_icp.paint_uniform_color([0.2, 0.2, 0.2])



# Visualize the aligned models
draw_registration_result(mesh, glb_icp, np.identity(4))

In [ ]:
print("ICP Transformation Matrix:")
print(icp_result.transformation)
print("ICP Fitness Score:")
print(icp_result.fitness)
print("ICP Inlier RMSE:")
print(icp_result.inlier_rmse)

In [ ]:
# pcd_df['Classification'].unique()

In [ ]:
# colors=np.zeros((len(pcd_df), 3))
# colors[pcd_df['Classification'] == 1] = [0.611, 0.8, 0.521]
# colors[pcd_df['Classification'] == 2] = [0.8, 0.670, 0.521]
# colors[pcd_df['Classification'] == 6] = [0.901, 0.419, 0.431]
# colors[pcd_df['Classification'] == 9] = [0.564, 0.850, 0.913]
# colors[pcd_df['Classification'] == 26] = [0.694, 0.662, 0.698]
# pcd_o3d.colors = o3d.utility.Vector3dVector(colors)